Restrucutred version of the same named script in the `scripts` directory. Used to understand the code better and analysze whats going on.

In [1]:
import copy

# isaac gym has to be imported before torch
from isaac_evaluation.grasp_quality_evaluation import GraspSuccessEvaluator

import scipy.spatial.transform
import numpy as np
from se3dif.datasets import AcronymGraspsDirectory
from se3dif.models.loader import load_model
from se3dif.samplers import ApproximatedGrasp_AnnealedLD, Grasp_AnnealedLD
from se3dif.utils import to_numpy, to_torch
from se3dif.visualization import grasp_visualization

import torch

Importing module 'gym_37' (/home/moritz/Documents/isaacgym/python/isaacgym/_bindings/linux-x86_64/gym_37.so)
Setting GYM_USD_PLUG_INFO_PATH to /home/moritz/Documents/isaacgym/python/isaacgym/_bindings/linux-x86_64/usd/plugInfo.json


/home/moritz/miniconda3/envs/alr/envs/se3dif_env/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


PyTorch version 1.13.1+cu116
Device count 1
/home/moritz/Documents/isaacgym/python/isaacgym/_bindings/src/gymtorch


Using /home/moritz/.cache/torch_extensions/py37_cu116 as PyTorch extensions root...
Emitting ninja build file /home/moritz/.cache/torch_extensions/py37_cu116/gymtorch/build.ninja...
Building extension module gymtorch...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)


ninja: no work to do.


Loading extension module gymtorch...


In [2]:
N_GRASPS = 10               # argument values from README: 10,10,10. default in argparse: 200
OBJ_ID = 0                  # argument values from README: 0,10,12.  default in argparse: 0
obj_class = "ScrewDriver"   # argument values from README: 'ScrewDriver','grasp_dif_mugs','Mug'. default in argparse: "Laptop"
n_envs = 30                 # hardcoded in the code with default value 30 in __main__
device = "cuda:0"           # argumant values from README: n.a., n.a., n.a. default in argparse: "cuda:0"
EVAL_SIM = False            # argument values from README: n.a., n.a., n.a. default in argparse: False
model = "grasp_dif_multi"   # argument values from README: n.a., 'grasp_dif_mugs', n.a. default in argparse: 'grasp_dif_multi'
batch = 10                  # hardcoded in code with default value 10 in get_approximated_grasp_diffusion_field

In [3]:
# def sample_pointcloud(obj_id=0, obj_class='Mug'):
#    ...
# P, mesh, trans, rot_quad = sample_pointcloud(obj_id, obj_class)

# get mesh of object and sample pointcloud
acronym_grasps = AcronymGraspsDirectory(data_type=obj_class)
mesh = acronym_grasps.avail_obj[OBJ_ID].load_mesh()
P = mesh.sample(1000)

# get a random rotation and apply it to the pointcloud
sampled_rot = scipy.spatial.transform.Rotation.random()
rot = sampled_rot.as_matrix()
rot_quat = sampled_rot.as_quat()

P = np.einsum('mn,bn->bm', rot, P)
P *= 8.
P_mean = np.mean(P, 0)
P += -P_mean

# recenter the pointcloud (??)
H = np.eye(4)
H[:3,:3] = rot
mesh.apply_transform(H)
mesh.apply_scale(8.) # ???
H = np.eye(4)
H[:3,-1] = -P_mean
mesh.apply_transform(H)
translational_shift = copy.deepcopy(H)


In [4]:
print("P: ", P.min(0), P.max(0), P.shape, P.dtype, P.mean(0))
print("mesh: ", type(mesh), mesh.vertices.min(0), mesh.vertices.max(0), mesh.vertices.shape, mesh.vertices.dtype, mesh.vertices.mean(0))
print("translational shift" , translational_shift)
print("rotational quat" , rot_quat)

P:  [-0.17545176 -0.3779445  -0.08673489] [0.13879214 0.78269891 0.07418472] (1000, 3) float64 [ 1.98521755e-17 -1.86156646e-16 -2.81719092e-18]
mesh:  <class 'trimesh.base.Trimesh'> [-0.17623076 -0.37902653 -0.08859384] [0.1414876  0.78394328 0.07529752] (789, 3) float64 [-0.0091348   0.03828545 -0.00031418]
translational shift [[ 1.          0.          0.         -0.03230393]
 [ 0.          1.          0.          0.15495371]
 [ 0.          0.          1.          0.00696618]
 [ 0.          0.          0.          1.        ]]
rotational quat [ 0.29161696 -0.66390253  0.67454917  0.13847884]


In [5]:
# generator, model = get_approximated_grasp_diffusion_field(P, args, device)
# def get_approximated_grasp_diffusion_field(p, args, device='cpu'):

## Load model
model_args = {
    'device': device,
    'pretrained_model': model
}
model = load_model(model_args)

context = to_torch(P[None,...], device)
model.set_latent(context, batch=batch)

########### 2. SET SAMPLING METHOD #############
generator = Grasp_AnnealedLD(model, batch=batch, T=70, T_fit=50, k_steps=2, device=device)

In [7]:
print("generator: ", type(generator))
print("model: ", type(model))


generator:  <class 'se3dif.samplers.grasp_samplers.Grasp_AnnealedLD'>
model:  <class 'se3dif.models.grasp_dif.GraspDiffusionFields'>


In [ ]:
H = generator.sample()

H_grasp = copy.deepcopy(H)
# counteract the translational shift of the pointcloud (as the spawned model in simulation will still have it)
H_grasp[:, :3, -1] = (H_grasp[:, :3, -1] - torch.as_tensor(translational_shift[:3,-1],device=device)).float()
H[..., :3, -1] *=1/8.
H_grasp[..., :3, -1] *=1/8.

## Visualize results ##

vis_H = H.squeeze()
P *=1/8
mesh = mesh.apply_scale(1/8)
grasp_visualization.visualize_grasps(to_numpy(H), p_cloud=P, mesh=mesh)

In [ ]:
if (EVAL_SIM):
    ## Evaluate Grasps in Simulation##
    num_eval_envs = 10
    evaluator = GraspSuccessEvaluator(obj_class, n_envs=num_eval_envs, idxs=[args.obj_id] * num_eval_envs, viewer=True, device=device, \
                                        rotations=[rot_quad]*num_eval_envs, enable_rel_trafo=False)
    succes_rate = evaluator.eval_set_of_grasps(H_grasp)
    print('Success cases : {}'.format(succes_rate))